##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TPU

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/guide/tpu" class=""><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" class="">在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/tpu.ipynb" class=""><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" class="">在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/tpu.ipynb" class=""><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" class="">在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/tpu.ipynb" class=""><img src="https://tensorflow.google.cn/images/download_logo_32px.png" class="">下载笔记本</a></td>
</table>

目前，Keras 和 Google Colab 已提供对 Cloud TPU 的实验性支持。在运行此 Colab 笔记本之前，请在以下路径下检查笔记本设置，确保硬件加速器为 TPU：Runtime > Change runtime type > Hardware accelerator > TPU。

## 设置

In [ ]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

## TPU 初始化

与运行用户 Python 程序的本地流程不同，TPU 通常位于 Cloud TPU 工作进程上。因此，需要完成一些初始化工作才能连接到远程集群并初始化 TPU。请注意，`TPUClusterResolver` 的 `tpu` 参数是一个仅适用于 Colab 的特殊地址。如果在 Google 计算引擎 (GCE) 上运行，应传入 CloudTPU 的名称。

注：必须将 TPU 初始化代码放在程序的开头位置。

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

## 手动设备放置

初始化 TPU 后，您可以通过手动设备放置将计算分配给单个 TPU 设备。


In [ ]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
with tf.device('/TPU:0'):
  c = tf.matmul(a, b)
print("c device: ", c.device)
print(c)

## 分布策略

大多数情况下，用户会希望以数据并行方式在多个 TPU 上运行模型。分布策略是一个抽象概念，可用于在 CPU、GPU 或 TPU 上驱动模型。如果简单地交换分布策略，模型将在指定设备上运行。有关详细信息，请参阅[分布策略指南](./distributed_training.ipynb)。

首先，创建 `TPUStrategy` 对象。

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

要复制计算，以便在所有 TPU 核心中运行，可以直接将其传递给 `strategy.run` API。在下面的示例中，所有核心都会获得相同的输入 `(a, b)`，并单独对每个核心执行矩阵乘法运算。输出是所有副本的值。

In [ ]:
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

z = strategy.run(matmul_fn, args=(a, b))
print(z)

## TPU 上的分类

我们已经学习了基本概念，现在来看看具体的示例。本指南会演示如何使用分布策略 `tf.distribute.experimental.TPUStrategy` 来驱动 Cloud TPU 和训练 Keras 模型。


### 定义 Keras 模型

下面是使用 Keras 的 MNIST 模型的定义，与您可能在 CPU 或 GPU 上使用的定义相同。请注意，需要将创建 Keras 模型的代码放在 `strategy.scope` 内，这样才能在每个 TPU 设备上创建变量。代码的其他部分不必放在策略作用域内。

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

### 输入数据集

使用 Cloud TPU 时，有效利用 `tf.data.Dataset` API 很关键，因为如果提供数据的速度不够快，则无法利用 Cloud TPU。有关数据集性能的详细信息，请参阅[输入流水线性能指南](./data_performance.ipynb)。

除了最简单的实验（使用 `tf.data.Dataset.from_tensor_slices` 或其他计算图中的数据）外，您都需要将数据集读取的所有数据文件存储在 Google Cloud Storage (GCS) 存储分区中。

对于大多数用例，建议将数据转换为 `TFRecord` 格式，并使用 `tf.data.TFRecordDataset` 进行读取。有关操作方法的详细信息，请参阅 [TFRecord 和 tf.Example 教程](../tutorials/load_data/tfrecord.ipynb)。不过，这并非硬性要求，如果您愿意，可以使用其他数据集读取器（`FixedLengthRecordDataset` 或 `TextLineDataset`）。

使用 `tf.data.Dataset.cache` 可以将小型数据集完整地加载到内存中。

无论使用哪一种数据格式，我们都强烈建议使用大文件（100MB 左右）。在这种网络化环境下，这一点尤其重要，因为打开文件的开销非常高。

这里应使用 `tensorflow_datasets` 模块获取 MNIST 训练数据的副本。请注意，代码中已指定 `try_gcs` 使用公共 GCS 存储分区中提供的副本。如果不这样指定，TPU 将无法访问下载的数据。 

In [ ]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage to have a
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so users don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

### 使用 Keras 高级别 API 训练模型

您可以只使用 Keras fit/compile API 训练模型。下面的示例并非特定于 TPU，如果您有多个 GPU，并且使用 `MirroredStrategy` 而不是 `TPUStrategy`，则可以编写与下面的示例相同的代码。要了解更多信息，请查阅 [Keras 分布训练](https://tensorflow.google.cn/tutorials/distribute/keras)教程。

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

batch_size = 200
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset, 
          validation_steps=validation_steps)

为了减少 Python 开销，同时最大限度提高 TPU 的性能，请尝试使用 `Model.compile` 的 **experimental** `experimental_steps_per_execution` 参数。在本例中，它可以将吞吐量提升约 50%：

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                experimental_steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

### 使用自定义训练循环训练模型

您还可以直接使用 `tf.function` 和 `tf.distribute` API 创建和训练模型。`strategy.experimental_distribute_datasets_from_function` API 用于从给定数据集函数分布数据集。请注意，在本例中，传递给数据集的批次大小是每个副本的批次大小，而非全局批次大小。要了解更多信息，请查阅[使用 tf.distribute.Strategy 进行自定义训练](https://tensorflow.google.cn/tutorials/distribute/custom_training)教程。


首先，创建模型、数据集和 tf.function。

In [ ]:
# Create the model, optimizer and metrics inside strategy scope, so that the
# variables can be mirrored on each device.
with strategy.scope():
  model = create_model()
  optimizer = tf.keras.optimizers.Adam()
  training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
  training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      'training_accuracy', dtype=tf.float32)

# Calculate per replica batch size, and distribute the datasets on each TPU
# worker.
per_replica_batch_size = batch_size // strategy.num_replicas_in_sync

train_dataset = strategy.experimental_distribute_datasets_from_function(
    lambda _: get_dataset(per_replica_batch_size, is_training=True))

@tf.function
def train_step(iterator):
  """The step function for one training step"""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits=True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  strategy.run(step_fn, args=(next(iterator),))

然后运行训练循环。

In [ ]:
steps_per_eval = 10000 // batch_size

train_iterator = iter(train_dataset)
for epoch in range(5):
  print('Epoch: {}/5'.format(epoch))

  for step in range(steps_per_epoch):
    train_step(train_iterator)
  print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))
  training_loss.reset_states()
  training_accuracy.reset_states()

### 在 `tf.function` 中利用多步法提高性能

在 `tf.function` 中运行多步可以提高性能。使用 `tf.range` 将 `strategy.run` 调用包装到 `tf.function` 内即可实现此目的。在 TPU 工作进程上，AutoGraph 会将其转换为 `tf.while_loop`。

在 `tf.function` 中，虽然多步法的性能更高，但是与单步法相比，可谓各有利弊。在 `tf.function` 中运行多步不够灵活，您无法在步骤中以 Eager 模式执行运算，也不能运行任意 Python 代码。


In [ ]:
@tf.function
def train_multiple_steps(iterator, steps):
  """The step function for one training step"""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits=True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  for _ in tf.range(steps):
    strategy.run(step_fn, args=(next(iterator),))

# Convert `steps_per_epoch` to `tf.Tensor` so the `tf.function` won't get 
# retraced if the value changes.
train_multiple_steps(train_iterator, tf.convert_to_tensor(steps_per_epoch))

print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))

## 后续步骤

- [Google Cloud TPU 文档](https://cloud.google.com/tpu/docs/) - 设置和运行 Google Cloud TPU。
- [使用 TensorFlow 进行分布式训练](./distributed_training.ipynb) - 如何利用分布策略以及演示最佳做法的许多示例的链接。
- [TensorFlow 官方模型](https://github.com/tensorflow/models/tree/master/official) - 与 Cloud TPU 兼容的最先进的 TensorFlow 2.x 模型示例。
- [Google Cloud TPU 性能指南](https://cloud.google.com/tpu/docs/performance-guide) - 通过为应用调整  Cloud TPU 配置参数来进一步提高 Cloud TPU 性能。